<a href="https://colab.research.google.com/github/jayanthchundru/GPT/blob/main/eng_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"

try:
  response = requests.get(url)
  response.raise_for_status() # if anything goes wrong "error"

  text_data = response.text
  print(f"Text data loaded successfuly from the url {url}")
  print(f"{text_data[:100]}")
except requests.expections.RequestException as e:
  print(f"Error in loading the text data {e}")


Text data loaded successfuly from the url https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [ ]:
chars = sorted(list(set(text_data)))
vocab_len = len(chars)

print(chars)
print(vocab_len)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65


In [ ]:
char_int = { ch:i for i , ch in enumerate(chars) }
print(char_int)
int_char = {i : ch for i, ch in enumerate(chars)}
print(int_char)


{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i',

In [ ]:
encode = lambda s: [char_int[c] for c in s]
decode = lambda i: [int_char[num] for num in i]

Character Level tokenizer

In [ ]:
print(encode('How are you'))

"".join(decode(encode('How are you')))

[20, 53, 61, 1, 39, 56, 43, 1, 63, 53, 59]


'How are you'

In [ ]:
import torch

data = torch.tensor(encode(text_data), dtype=torch.long)

print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [ ]:
print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [ ]:
n = int(0.9 * len(data))
print(n)

train_data = data[:n]
val_data = data[n:]

1003854


In [ ]:
context_length = 15

x = train_data[:context_length+1]
print(x)

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14])


In [ ]:
y = train_data[1:context_length+1]

for i in range(context_length):
  context = x[:i+1]
  target = y[i]
  print(f"For input {context} : next target {target}")

For input tensor([18]) : next target 47
For input tensor([18, 47]) : next target 56
For input tensor([18, 47, 56]) : next target 57
For input tensor([18, 47, 56, 57]) : next target 58
For input tensor([18, 47, 56, 57, 58]) : next target 1
For input tensor([18, 47, 56, 57, 58,  1]) : next target 15
For input tensor([18, 47, 56, 57, 58,  1, 15]) : next target 47
For input tensor([18, 47, 56, 57, 58,  1, 15, 47]) : next target 58
For input tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]) : next target 47
For input tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47]) : next target 64
For input tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64]) : next target 43
For input tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43]) : next target 52
For input tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52]) : next target 10
For input tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10]) : next target 0
For input tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0])

In [ ]:
torch.manual_seed(1340)

batch_size = 4
block_size = 15

def get_batch(split):
  data = train_data if split == "train" else val_data
  ix = torch.randint(len(data)- block_size,(batch_size,))
  x = torch.stack([data[i : i+block_size] for i in ix])
  y = torch.stack([data[i+1 : i + block_size + 1]for i in ix])

  return x,y

xb, yb = get_batch("train")
print(f"Inputs : {xb}")
print(f"Input Shape : {xb.shape}")
print('\n')
print(f"Targets : {yb}")
print(f"Target Shape : {yb.shape}")

Inputs : tensor([[57, 47, 56,  6,  1, 58, 46, 39, 52,  1, 42, 43, 52, 63, 47],
        [59, 56,  1, 42, 47, 60, 53, 56, 41, 43,  6,  1, 63, 53, 59],
        [57, 46, 39, 50, 50,  1, 39, 57, 57, 59, 56, 43,  1, 51, 63],
        [58, 46, 47, 52, 45,  1, 57, 43, 47, 64, 43, 42,  0, 20, 47]])
Input Shape : torch.Size([4, 15])


Targets : tensor([[47, 56,  6,  1, 58, 46, 39, 52,  1, 42, 43, 52, 63, 47, 52],
        [56,  1, 42, 47, 60, 53, 56, 41, 43,  6,  1, 63, 53, 59, 52],
        [46, 39, 50, 50,  1, 39, 57, 57, 59, 56, 43,  1, 51, 63,  1],
        [46, 47, 52, 45,  1, 57, 43, 47, 64, 43, 42,  0, 20, 47, 57]])
Target Shape : torch.Size([4, 15])
